In [1]:
# Import required libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from pymongo import MongoClient
from collections import Counter, defaultdict
import os
import warnings
import sys
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

PLOTS_PATH = os.path.join(library_path, 'plots')

print("Libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")

Libraries imported successfully!
Current working directory: /home/luis/CGE/voice_db/analysis_notebooks


In [2]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["Diagnosis_Severity_PD_Voice"]
collection = db["studies"]

print("🔄 Loading studies from MongoDB...")
fields_to_extract = {
    "doi"             : 1, 
    "year"            : 1, 
    "study_id"        : 1,
    "problem"         : 1,
    "ml_problem_type" : 1,
    '_id'             : 0
}  # 1 = include, 0 = exclude
studies_cursor = collection.find({}, fields_to_extract)
studies_list = list(studies_cursor)


print(f"📊 Total studies loaded: {len(studies_list)}")
print(f"📄 Sample document keys: {list(studies_list[0].keys()) if studies_list else 'No documents found'}")

🔄 Loading studies from MongoDB...
📊 Total studies loaded: 381
📄 Sample document keys: ['year', 'doi', 'study_id', 'ml_problem_type', 'problem']


In [3]:
studies_df = pd.DataFrame(studies_list)
studies_df.head()

,year,doi,study_id,ml_problem_type,problem
0,2020,10.1109/ACCESS.2020.2974008,10.1109/ACCESS.2020.2974008_1,[Classification: Binary ],Parkinson's disease diagnosis
1,2020,10.2196/18689,10.2196/18689_1,[Classification: Binary ],Parkinson's disease diagnosis
2,2020,10.3233/JIFS-179714,10.3233/JIFS-179714_1,[Classification: Binary ],Parkinson's disease diagnosis
3,2020,10.1016/j.bbe.2019.05.005,10.1016/j.bbe.2019.05.005_2,[Classification: Binary ],Parkinson's disease diagnosis
4,2020,10.15439/2020F188,10.15439/2020F188_1,[Classification: Binary ],Parkinson's disease diagnosis


In [4]:
studies_df = studies_df.explode('ml_problem_type')
studies_df.shape

(382, 5)

In [5]:
ml_problem_df =studies_df.groupby(by=['doi', 'ml_problem_type']).agg({'year': 'first'}).reset_index()
ml_problem_df.head()

,doi,ml_problem_type,year
0,10.1002/cpe.6419,Classification: Binary,2021
1,10.1002/eng2.13091,Classification: Binary,2025
2,10.1007/978-3-030-31129-2_15,Regression,2020
3,10.1007/978-3-030-82153-1_34,Classification: Binary,2021
4,10.1007/978-3-030-86993-9_44,Classification: Binary,2021


In [6]:
ml_problem_df['ml_problem_type'].value_counts(normalize=True) * 100

ml_problem_type
Classification: Binary        89.056604
Regression                     8.301887
Classification: Multiclass     2.641509
Name: proportion, dtype: float64